In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from src.database import VastDB
from src.manager import DbManager
from src.fetch import fetch
from src.preprocess import preprocess, split_raw
from src import const
import logging


logging.basicConfig(format=const.LOG_FORMAT,
                    level=logging.DEBUG,
                    datefmt='%d-%m-%Y %I:%M:%S')

# logger = 
# logger.setLevel(logging.DEBUG)

In [3]:
raw = fetch()
preprocess(raw)
machines, offers = split_raw(raw)

[22-09-2023 05:45:49] [INFO] [API] fetching data from '500.farm'
[22-09-2023 05:45:49] [DEBUG] Starting new HTTPS connection (1): 500.farm:443
[22-09-2023 05:45:50] [DEBUG] https://500.farm:443 "GET /vastai-exporter/offers HTTP/1.1" 200 None


In [8]:
str((1, 2)).strip()

'(1, 2)'

In [9]:
db = VastDB('../data/test.db')
db.connect()

[22-09-2023 06:12:42] [INFO] Connected to the database '../data/test.db'


In [10]:
db.manager.get_tables()

['timestamp_idx',
 'tmp_machines',
 'tmp_offers',
 'host_mach_map',
 'mach_offer_map']

In [14]:
info = db.manager.get_tbl_info('tmp_machines')
'host_id' in info['name']

True

In [12]:
db.close()

[20-09-2023 08:38:17] [INFO] Disconnected from the database '../data/test.db'.


In [61]:
db.manager.delete_table('tmp_machines')
db.manager.delete_table('tmp_offers')
db.manager.delete_table('host_mach_map')
db.manager.delete_table('mach_offr_map')

[21-09-2023 06:25:25] [INFO] Table 'tmp_machines' deleted
[21-09-2023 06:25:25] [INFO] Table 'tmp_offers' deleted


In [31]:
df = db.manager.table_to_df('tmp_machines')
df.values.tolist()

[[1, 2, 16], [1, 3, 16], [2, 4, 16]]

In [4]:
db.manager.create_table('tmp_machines', ['host_id', 'machine_id', 'timestamp'])
db.manager.create_table('tmp_offers', ['machine_id', 'id', 'timestamp'])

[21-09-2023 06:28:53] [INFO] Table 'tmp_machines' created
[21-09-2023 06:28:53] [INFO] Table 'tmp_offers' created


In [5]:
db.manager.insert('tmp_machines', 
                  [[1, 2, 16], 
                   [1, 3, 16], 
                   [2, 4, 16]])
db.manager.insert('tmp_offers', 
                  [[2, 1, 16], 
                   [3, 2, 16], 
                   [4, 3, 16]])

[21-09-2023 06:28:57] [INFO] 3 Rows inserted into 'tmp_machines'
[21-09-2023 06:28:57] [INFO] 3 Rows inserted into 'tmp_offers'


In [15]:
db.close()

[21-09-2023 05:56:15] [INFO] Disconnected from the database '../data/test.db'.


In [9]:
db.create_map_tables()
db.update_tables()
db.manager.commit()
db.manager.get_db_stats()

,name,pages,size,percentage
0,total,6,0.02,100.00
1,sqlite_schema,1,0.00,19.53
2,timestamp_idx,1,0.00,19.53
3,tmp_machines,1,0.00,19.53
4,tmp_offers,1,0.00,19.53
5,host_mach_map,1,0.00,19.53
6,mach_offer_map,1,0.00,19.53


In [8]:
db.manager.table_to_df('host_mach_map')

,machine_id,host_id,timestamp
0,2,1,16
1,3,1,16
2,4,2,16


In [14]:
db.update_tables()
db.manager.commit()

In [16]:
DbManager().conn

In [15]:
db.create_tmp_tables(machines, offers)
db.create_map_tables()

In [16]:
db.manager.commit()
db.manager.get_db_stats()
# db.manager.delete_table('int32')

,name,pages,size,percentage
0,total,302,1.18,100.00
1,tmp_offers,194,0.76,64.22
2,tmp_machines,80,0.31,26.48
3,offer_mach_map,18,0.07,5.96
4,mach_host_map,8,0.03,2.65
5,sqlite_schema,1,0.00,0.33
6,timestamp_idx,1,0.00,0.33


In [18]:
db.create_ts_index()

In [19]:
db.update_tables()

0

In [20]:
db.manager.commit()
db.manager.get_db_stats()

,name,pages,size,percentage
0,total,302,1.18,100.00
1,tmp_offers,194,0.76,64.22
2,tmp_machines,80,0.31,26.48
3,offer_mach_map,18,0.07,5.96
4,mach_host_map,8,0.03,2.65
5,sqlite_schema,1,0.00,0.33
6,timestamp_idx,1,0.00,0.33


In [8]:
db.manager.table_to_df('offer_mach_map')
# db.manager.table_to_df('mach_offer_map').sort_values(by='machine_id')

,id,machine_id
0,5120551,4454
1,5120986,4378
2,5120988,4379
3,5120991,4381
4,5120996,4458
...,...,...
3940,7058112,12769
3941,7058113,12769
3942,7058124,9617
3943,7058125,9617
